# End-to-end prediction of (synthetic) transcriptome from sparse genes

## Imports

In [ ]:
import numpy as np
import pandas as pd
import scanpy.api as sc

from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.preprocessing import scale

from scipy.stats import pearsonr, spearmanr, kendalltau
from statsmodels.stats.moment_helpers import cov2corr

import geneselection.solvers.elasticnet.pca as epca
import geneselection.solvers.elasticnet.utils as eutils
from geneselection.datasets.correlated_random_variables import CorrelatedNormal
import geneselection.datasets.scrnaseq_cardio_20181129 as cardio
from geneselection.utils.data import tidy

import altair as alt
alt.data_transformers.enable("default", max_rows=None)

## Generate synthetic data with same covariance as cardio diff data

In [ ]:
# adata_cardio = cardio.load()
# adata_cardio = eutils.preprocess_cardio(adata_cardio)
# adata_cardio = eutils.subset_cardio(adata_cardio, days=['D12', 'D14'])

In [ ]:
# import torch
# import torch.nn.functional as F

# X = torch.from_numpy(adata_cardio.X.astype(np.float64))
# X = X - X.mean(dim=0)
# X = X.cuda(0)

# c = torch.matmul(X.t(),X)
# k = torch.diag(1/torch.sqrt(torch.diag(c)))
# Sigma = k @ c @ k
# Sigma = Sigma.cpu().numpy()

In [ ]:
# correlated_normal = CorrelatedNormal(Sigma=Sigma)

# adata_all = correlated_normal.sample(20000)
# adata_all.X = adata_all.X.astype(np.float64)
# adata_all.var.index = adata_cardio.var.index

In [ ]:
# adata_all.write_h5ad("synthetic_data_empirical_cov_D12+D14.h5ad")

In [ ]:
adata_all = sc.read_h5ad("synthetic_data_empirical_cov_D12+D14.h5ad")

### Split off a train and test set

In [ ]:
adata = adata_all[:15000,:].copy()
adata_test = adata_all[15000:,:].copy()

## Select predictive genes with elastic net PCA

In [ ]:
pca = PCA(n_components=25, svd_solver="randomized")
pca.fit(adata.X)

In [ ]:
df = pd.DataFrame(list(enumerate(pca.explained_variance_ratio_)))
df.columns = ["PC", "Explained Variance Ratio"]

alt.Chart(df).mark_point().encode(
    x=alt.X("PC"),
    y=alt.Y("Explained Variance Ratio", scale=alt.Scale(type='log')),
)

### Parameters

In [ ]:
params = dict(lambda_path = np.geomspace(10, 0.01, num=100),   # lambda path
              alpha = 0.9,                                     # fraction of regularization devoted to L1 prenalty
              n_pcs = 5,                                       # number of pcs to predit with multitask elastic net
              pc_weights = "scaled",                           # relative importance in predicting pcs
              n_bootstraps = 100,                              # number of bootstrap replicates
              n_processes = 25,                                # number of parallel processes to use
              thresholds = np.linspace(0.01, 1, num=100))      # selection thresholds for including genes

### Run bootstrap replicates

In [ ]:
results = epca.parallel_runs(adata,
                             n_processes=params["n_processes"],
                             n_bootstraps=params["n_bootstraps"],
                             n_pcs=params["n_pcs"],
                             alpha=params["alpha"],
                             lambda_path=params["lambda_path"],
                             pc_weights=params["pc_weights"])

### Inspect results

In [ ]:
eutils.thresh_lambda_plot(results,
                          adata,
                          thresholds=params["thresholds"],
                          lambdas=params["lambda_path"])

In [ ]:
# eutils.hub_persistence_plot(adata, results)

### Pick maximally informative sparse genes

In [ ]:
predictive_genes = eutils.get_selected_genes(results,
                                             adata,
                                             lambda_index=50,
                                             selection_threshold_index=90,
                                             thresholds=params["thresholds"])

In [ ]:
len(predictive_genes)

In [ ]:
predictive_genes = [p + "_HUMAN" for p in predictive_genes]
# predictive_genes

### See how well we predict PCs using selected genes

#### Fit pca to training data

In [ ]:
pca = PCA(n_components=params["n_pcs"], svd_solver="randomized")
pca.fit(adata.X)

#### pcs are our targets to regress

In [ ]:
y_train = pca.transform(adata.X)
y_train = scale(y_train)

#### regressors are the sparse genes

In [ ]:
X_train = adata[:,predictive_genes].X

#### fit the regression

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

#### predict on the training data

In [ ]:
y_pred_train = reg.predict(X_train)

#### predict on the test and train data

In [ ]:
y_test = pca.transform(adata_test.X)
y_test = scale(y_test)

X_test = adata_test[:,predictive_genes].X
y_pred_test = reg.predict(X_test)

#### organize data

In [ ]:
df_test_real = tidy(y_test).loc[:,1:]
df_test_real.columns = ["PC", "Real Value"]
df_test_real["PC"] += 1

df_test_pred = tidy(y_pred_test).loc[:,1:]
df_test_pred.columns = ["PC", "Predicted Value"]
df_test_pred["PC"] += 1

df_test = pd.concat([df_test_real, df_test_pred], axis=1)
df_test = df_test.iloc[:,[0,1,3]]
df_test["Split"] = "Test"

df_train_real = tidy(y_train).loc[:,1:]
df_train_real.columns = ["PC", "Real Value"]
df_train_real["PC"] += 1

df_train_pred = tidy(y_pred_train).loc[:,1:]
df_train_pred.columns = ["PC", "Predicted Value"]
df_train_pred["PC"] += 1

df_train = pd.concat([df_train_real, df_train_pred], axis=1)
df_train = df_train.iloc[:,[0,1,3]]
df_train["Split"] = "Train"

df_pc = pd.concat([df_test, df_train], ignore_index=True)

In [ ]:
alt.Chart(df_pc[df_pc["PC"] <= 4].sample(10000), width=200, height=200).mark_circle(size=10).encode(
    x='Real Value',
    y='Predicted Value'
).facet(
    column='Split:N',
    row='PC:N'
)

In [ ]:
for i in sorted(df_pc.PC.unique()):
    print(i, pearsonr(y_test[:,i-1],y_pred_test[:,i-1])[0])

## Model rest of genes using sparse gene set

### Subsets of to predict

In [ ]:
target_genes = np.array([g for g in adata.var.index if g not in predictive_genes])

### Fit regression

In [ ]:
y_train = adata[:,target_genes].X
y_test = adata_test[:,target_genes].X

In [ ]:
ridge = Ridge(alpha=1.0, tol=0.00001)
ridge.fit(X_train, y_train)

y_pred_all_train = ridge.predict(X_train)
y_pred_all_test = ridge.predict(X_test)

### Organize results

In [ ]:
perf_train = np.array([pearsonr(y_pred_all_train[:,i], y_train[:,i])[0] for i in range(y_train.shape[1])])
perf_test = np.array([pearsonr(y_pred_all_test[:,i], y_test[:,i])[0] for i in range(y_test.shape[1])])

df_perf_train = adata[:,target_genes].var.copy()
df_perf_train["Pearson Correlation"] = perf_train
df_perf_train["Split"] = "Train"

df_perf_test = adata[:,target_genes].var.copy()
df_perf_test["Pearson Correlation"] = perf_test
df_perf_test["Split"] = "Test"

df_perf = pd.concat([df_perf_train, df_perf_test])

### Correlation histogram

In [ ]:
alt.Chart(df_perf, height=300, width=400).mark_area(
    opacity=0.5,
    interpolate="step"
).encode(
    alt.X("Pearson Correlation", bin=alt.Bin(maxbins=100), scale=alt.Scale(domain=[0, 1])),
    alt.Y("count()", stack=None),
).facet(
    column="Split"
)

### See how well imputed genes recapitulate pcs

In [ ]:
pca_targ_genes = PCA(n_components=params["n_pcs"], svd_solver="randomized")
pca_targ_genes.fit(y_train)

In [ ]:
pcs_targ_genes_train = pca_targ_genes.transform(y_train)
pcs_targ_genes_train = scale(pcs_targ_genes_train)
pcs_targ_genes_test = pca_targ_genes.transform(y_test)
pcs_targ_genes_test = scale(pcs_targ_genes_test)

pcs_targ_genes_train_imputed = pca_targ_genes.transform(y_pred_all_train)
pcs_targ_genes_train_imputed = scale(pcs_targ_genes_train_imputed)
pcs_targ_genes_test_imputed = pca_targ_genes.transform(y_pred_all_test)
pcs_targ_genes_test_imputed = scale(pcs_targ_genes_test_imputed)

In [ ]:
df_test_real = tidy(pcs_targ_genes_test).loc[:,1:]
df_test_real.columns = ["PC", "Real Value"]
df_test_real["PC"] += 1

df_test_pred = tidy(pcs_targ_genes_test_imputed).loc[:,1:]
df_test_pred.columns = ["PC", "Predicted Value"]
df_test_pred["PC"] += 1

df_test = pd.concat([df_test_real, df_test_pred], axis=1)
df_test = df_test.iloc[:,[0,1,3]]
df_test["Split"] = "Test"

df_train_real = tidy(pcs_targ_genes_train).loc[:,1:]
df_train_real.columns = ["PC", "Real Value"]
df_train_real["PC"] += 1

df_train_pred = tidy(pcs_targ_genes_train_imputed).loc[:,1:]
df_train_pred.columns = ["PC", "Predicted Value"]
df_train_pred["PC"] += 1

df_train = pd.concat([df_train_real, df_train_pred], axis=1)
df_train = df_train.iloc[:,[0,1,3]]
df_train["Split"] = "Train"

df_pc_imp = pd.concat([df_test, df_train], ignore_index=True)

In [ ]:
alt.Chart(df_pc_imp[df_pc_imp["PC"] <= 4].sample(10000), width=200, height=200).mark_circle(size=10).encode(
    x='Real Value',
    y='Predicted Value'
).facet(
    column='Split:N',
    row='PC:N',
)

In [ ]:
for i in sorted(df_pc_imp.PC.unique()):
    print(i, pearsonr(pcs_targ_genes_test[:,i-1],pcs_targ_genes_test_imputed[:,i-1])[0])

### Use random subsets of genes = predictors as baseline

In [ ]:
N_trials = 101
test_results_random = np.zeros([len(adata.var) - len(predictive_genes), N_trials])
train_results_random = np.zeros([len(adata.var) - len(predictive_genes), N_trials])

for i in tqdm(range(N_trials)):
    predictive_genes_random = np.random.choice(adata.var.index, len(predictive_genes), replace=False)
    target_genes_random = np.array([g for g in adata.var.index if g not in predictive_genes_random])

    X_train_random = adata[:,predictive_genes_random].X
    X_test_random = adata_test[:,predictive_genes_random].X
    y_train_random = adata[:,target_genes_random].X
    y_test_random = adata_test[:,target_genes_random].X

    ridge = Ridge(alpha=1.0, tol=0.00001)
    ridge.fit(X_train_random, y_train_random)

    y_pred_all_train_random = ridge.predict(X_train_random)
    y_pred_all_test_random = ridge.predict(X_test_random)

    perf_train_random = np.array([pearsonr(y_pred_all_train_random[:,i], y_train_random[:,i])[0] for i in range(y_train_random.shape[1])])
    perf_test_random = np.array([pearsonr(y_pred_all_test_random[:,i], y_test_random[:,i])[0] for i in range(y_test_random.shape[1])])
    
    test_results_random[:,i] = perf_test_random
    train_results_random[:,i] = perf_train_random

In [ ]:
df_random_train = pd.DataFrame({"Median Random":np.median(train_results_random, axis=1), "Elastic Net":perf_train})
df_random_train["Split"] = "Train"
df_random_test = pd.DataFrame({"Median Random":np.median(test_results_random, axis=1), "Elastic Net":perf_test})
df_random_test["Split"] = "Test"
df_random_test
df_random = pd.concat([df_random_train, df_random_test])

In [ ]:
alt.Chart(df_random, width=400, height=400).mark_circle(size=10, opacity=0.25).encode(
    x=alt.X('Median Random', scale=alt.Scale(domain=[0, 1])),
    y=alt.Y('Elastic Net', scale=alt.Scale(domain=[0, 1]))
).facet(
    column='Split:N'
)

In [ ]:
df_random["Improvement Ratio"] = (df_random["Elastic Net"] - df_random["Median Random"])/df_random["Median Random"]

In [ ]:
alt.Chart(df_random, height=300, width=400).mark_area(
    opacity=0.5,
    interpolate="step"
).encode(
    alt.X("Improvement Ratio", bin=alt.Bin(maxbins=100), scale=alt.Scale(domain=[-1, 4])),
    alt.Y("count()", stack=None),
).facet(
    column="Split"
)

In [ ]:
np.mean(df_random[df_random["Split"] == "Test"]["Improvement Ratio"])

In [ ]:
np.median(df_random[df_random["Split"] == "Test"]["Improvement Ratio"])

### Reconstruct PCs from imputed genes and compare to both predicted and real

In [ ]:
y_pred_all_test_random.shape

In [ ]:
y_test.shape